In [49]:
import matplotlib.pyplot as plt
from dataload import *
from gan_architecture import *
from utils import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from dataload import *
from gan_architecture import *
import pandas as pd
import numpy as np
import copy

In [50]:
class SuperNetwork(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.lin = nn.Linear(input_size, 4)
        self.bn = nn.BatchNorm1d(4)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.lin(x)
        x = self.bn(x)
        x = self.activation(x)
        return x

In [51]:
bs = 16
input = torch.full((bs, 4), 1.5, dtype=torch.float, device='cpu')

In [52]:
s = SuperNetwork(4)
s2 = copy.deepcopy(s)
criterion = nn.BCELoss() 
optimizer = optim.Adam(s.parameters(), lr = 0.002)
optimizer2 = optim.Adam(s2.parameters(), lr = 0.002)

In [53]:
print(s.parameters)

<bound method Module.parameters of SuperNetwork(
  (lin): Linear(in_features=4, out_features=4, bias=True)
  (bn): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation): ReLU()
)>


In [54]:
loss = criterion(s(input), input)
loss2 = criterion(s2(input), input)
loss.backward()
loss2.backward()

In [55]:
for param in s.parameters():
  print(param.data)

for param in s2.parameters():
  print(param.data)

tensor([[ 0.2825,  0.0751,  0.3275, -0.3938],
        [-0.2841,  0.0759,  0.1744,  0.1236],
        [-0.4762, -0.4474, -0.0302, -0.1469],
        [-0.3312,  0.2187,  0.3164,  0.1254]])
tensor([-0.3025,  0.3563, -0.1819,  0.3387])
tensor([1., 1., 1., 1.])
tensor([0., 0., 0., 0.])
tensor([[ 0.2825,  0.0751,  0.3275, -0.3938],
        [-0.2841,  0.0759,  0.1744,  0.1236],
        [-0.4762, -0.4474, -0.0302, -0.1469],
        [-0.3312,  0.2187,  0.3164,  0.1254]])
tensor([-0.3025,  0.3563, -0.1819,  0.3387])
tensor([1., 1., 1., 1.])
tensor([0., 0., 0., 0.])


In [56]:
gen = s.modules()
next(gen)
m_list  = list(gen)

gen = s2.modules()
next(gen)
m_list2 = list(gen)


for l in m_list[:-1] :
    print(l.weight)

for l in m_list2[:-1] :
    print(l.weight)

Parameter containing:
tensor([[ 0.2825,  0.0751,  0.3275, -0.3938],
        [-0.2841,  0.0759,  0.1744,  0.1236],
        [-0.4762, -0.4474, -0.0302, -0.1469],
        [-0.3312,  0.2187,  0.3164,  0.1254]], requires_grad=True)
Parameter containing:
tensor([1., 1., 1., 1.], requires_grad=True)
Parameter containing:
tensor([[ 0.2825,  0.0751,  0.3275, -0.3938],
        [-0.2841,  0.0759,  0.1744,  0.1236],
        [-0.4762, -0.4474, -0.0302, -0.1469],
        [-0.3312,  0.2187,  0.3164,  0.1254]], requires_grad=True)
Parameter containing:
tensor([1., 1., 1., 1.], requires_grad=True)


In [57]:
apply_weight_decay(*m_list, weight_decay_factor=0.005, wo_bn=True)

In [58]:
optimizer.step()
optimizer2.step()

In [59]:
for l in m_list[:-1] :
    print(l.weight)

for l in m_list2[:-1] :
    print(l.weight)

Parameter containing:
tensor([[ 0.2805,  0.0731,  0.3255, -0.3918],
        [-0.2821,  0.0739,  0.1724,  0.1216],
        [-0.4742, -0.4454, -0.0282, -0.1449],
        [-0.3292,  0.2167,  0.3144,  0.1234]], requires_grad=True)
Parameter containing:
tensor([0.9980, 0.9980, 0.9980, 0.9980], requires_grad=True)
Parameter containing:
tensor([[ 0.2825,  0.0751,  0.3275, -0.3938],
        [-0.2841,  0.0759,  0.1744,  0.1236],
        [-0.4762, -0.4474, -0.0302, -0.1469],
        [-0.3312,  0.2187,  0.3164,  0.1254]], requires_grad=True)
Parameter containing:
tensor([1., 1., 1., 1.], requires_grad=True)
